In [36]:
"""
CNN for Smartphone-Based Recognition of Human Activities and Postural Transitions Data Set 
Link: http://archive.ics.uci.edu/ml/datasets/Smartphone-Based+Recognition+of+Human+Activities+and+Postural+Transitions

Using Raw Data folder and splitting into train and testing with k-fold validation.

***********
Overall data formatting:
         accX  accY  accZ  gyroX  gyroY  gyroZ
Samples
1
2
3
...
n
************

1. Look into 1D convolutions
2. Figure out loading in data efficiently, possibly use some library
3. Samples have differing row lengths (due to time), truncate to same length
    don't know if its necessary. 
    SOLVED: The action/label lengths for each action are chopped up blocks
    of time of entire experiment sampled data length.
    Must divide up and chop full experiment sample data into blocks of time
    that correspond to action/label length from labels .txt file
"""

"\nCNN for Smartphone-Based Recognition of Human Activities and Postural Transitions Data Set \nLink: http://archive.ics.uci.edu/ml/datasets/Smartphone-Based+Recognition+of+Human+Activities+and+Postural+Transitions\n\nUsing Raw Data folder and splitting into train and testing with k-fold validation.\n\n***********\nOverall data formatting:\n         accX  accY  accZ  gyroX  gyroY  gyroZ\nSamples\n1\n2\n3\n...\nn\n************\n\n1. Look into 1D convolutions\n2. Figure out loading in data efficiently, possibly use some library\n3. Samples have differing row lengths (due to time), truncate to same length\n    don't know if its necessary. \n    SOLVED: The action/label lengths for each action are chopped up blocks\n    of time of entire experiment sampled data length.\n    Must divide up and chop full experiment sample data into blocks of time\n    that correspond to action/label length from labels .txt file\n"

In [37]:
import numpy as np
import glob

classes = ('WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING', 'STANDING', 'LAYING', 'STAND_TO_SIT', 'SIT_TO_STAND', 'SIT_TO_LIE', 'LIE_TO_SIT', 'STAND_TO_LIE', 'LIE_TO_STAND')

# labels array
labels = np.loadtxt('HAPT_Data_Set/RawData/labels.txt')
# Separate labels for each action contained within each experiment, create array 

# import .txt files of samples with data
ACCtxt = glob.glob("HAPT_Data_Set/RawData/ACC/*.txt")
GYROtxt = glob.glob("HAPT_Data_Set/RawData/Gyro/*.txt")
ACCexpData = []
GYROexpData = []

# Create multidemensional arrays comprised of samples of each experiment
for exp in enumerate(ACCtxt):
    ACCexpData.append(np.loadtxt(ACCtxt[exp[0]]))

for expG in enumerate(GYROtxt):
    GYROexpData.append(np.loadtxt(GYROtxt[expG[0]]))

In [59]:
# Convert to np arrays for ease of manipulation
ACCexp = np.asarray(ACCexpData)
GYROexp = np.asarray(GYROexpData)

# Go through labels array and create new arrays based on labels for each experiment
labelArray = []
expArray = []
expcount = 1
for row in labels:
    exp = int(row[0])
    if exp==expcount:
        expArray.append(row)
    else:
        expArray = np.asarray(expArray)
        labelArray.append(expArray)
        expArray = []
        expArray.append(row)
        expcount+=1
    
# Now, we have label array according to experiment number and comprised of time slices of each
# action with its respective label.
npLabels = np.asarray(labelArray)
print(npLabels.shape)
print(npLabels[0].shape)
print(npLabels[0])

(60,)
(22, 5)
[[1.0000e+00 1.0000e+00 5.0000e+00 2.5000e+02 1.2320e+03]
 [1.0000e+00 1.0000e+00 7.0000e+00 1.2330e+03 1.3920e+03]
 [1.0000e+00 1.0000e+00 4.0000e+00 1.3930e+03 2.1940e+03]
 [1.0000e+00 1.0000e+00 8.0000e+00 2.1950e+03 2.3590e+03]
 [1.0000e+00 1.0000e+00 5.0000e+00 2.3600e+03 3.3740e+03]
 [1.0000e+00 1.0000e+00 1.1000e+01 3.3750e+03 3.6620e+03]
 [1.0000e+00 1.0000e+00 6.0000e+00 3.6630e+03 4.5380e+03]
 [1.0000e+00 1.0000e+00 1.0000e+01 4.5390e+03 4.7350e+03]
 [1.0000e+00 1.0000e+00 4.0000e+00 4.7360e+03 5.6670e+03]
 [1.0000e+00 1.0000e+00 9.0000e+00 5.6680e+03 5.8590e+03]
 [1.0000e+00 1.0000e+00 6.0000e+00 5.8600e+03 6.7860e+03]
 [1.0000e+00 1.0000e+00 1.2000e+01 6.7870e+03 6.9770e+03]
 [1.0000e+00 1.0000e+00 1.0000e+00 7.4960e+03 8.0780e+03]
 [1.0000e+00 1.0000e+00 1.0000e+00 8.3560e+03 9.2500e+03]
 [1.0000e+00 1.0000e+00 1.0000e+00 9.6570e+03 1.0567e+04]
 [1.0000e+00 1.0000e+00 1.0000e+00 1.0750e+04 1.1714e+04]
 [1.0000e+00 1.0000e+00 3.0000e+00 1.3191e+04 1.3846e+04]


In [58]:
# Now, we must split the ACC and GYRO experiment data into a multidimensional array 
# that corresponds to labelled action for particular time splices.
print(ACCexp[0].shape)
#for experiment in

# below is how i will access the time splices, simply grab time splices for each
# action of npLabels array and use variables to access them within ACC/GYRO data
# WOOHOO makin' progress!
print(ACCexp[0][250:1232])

(21303, 3)
[[ 1.0375     -0.30000001 -0.30833334]
 [ 1.09444447 -0.32500002 -0.36527779]
 [ 1.04861114 -0.32777778 -0.33750002]
 ...
 [ 0.97638895 -0.18333334 -0.2888889 ]
 [ 0.97361117 -0.18194445 -0.28750001]
 [ 0.97638895 -0.18472223 -0.29583334]]
